In [1]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [7]:
def get_transformation_matrix(animal):
    try:
        #url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        url = f'http://localhost:8000/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix

In [4]:
prep_id = 'DK52'

In [8]:
transformation = get_transformation_matrix(prep_id)
r = np.array(transformation['rotation'])
t = np.array(transformation['translation'])

In [9]:
print(r)
print(t)

[[ 0.90823119 -0.24441544 -0.05697743]
 [ 0.2488724   0.90471918  0.08611017]
 [ 0.03237074 -0.09804858  0.93659374]]
[[-15509.17500957]
 [-12625.10609468]
 [   -66.471405  ]]


In [10]:
person_id = 1 # me
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .filter(CenterOfMass.input_type == 'manual')\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section]

In [11]:
def brain_to_atlas_transform(
    brain_coord, r, t,
    brain_scale=(0.325, 0.325, 20),
    atlas_scale=(10, 10, 20)
):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates.
    
    The recorded r, t is the transformation from brain to atlas such that:
        t_phys = atlas_scale @ t
        atlas_coord_phys = r @ brain_coord_phys + t_phys
        
    Currently we erraneously have:
        t_phys = brain_scale @ t
    This should be fixed in the future.
    """
    brain_scale = np.diag(brain_scale)
    atlas_scale = np.diag(atlas_scale)

    # Transform brain coordinates to physical space
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    brain_coord_phys = brain_scale @ brain_coord
    
    # Apply affine transformation in physical space
    # t_phys = atlas_scale @ t
    # The following is wrong but it is what the rest of the code assumes
    # We need to fix it in the future
    # this gets flip flopped to above line
    t_phys = brain_scale @ t
    atlas_coord_phys = r @ brain_coord_phys + t_phys

    # Bring atlas coordinates back to atlas space
    atlas_coord = np.linalg.inv(atlas_scale) @ atlas_coord_phys

    return atlas_coord.T[0] # Convert back to a row vector

In [15]:
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    brain_coords = np.asarray([x, y, section])
    transformed = brain_to_atlas_transform(brain_coords, r, t)
    x = transformed[0]
    y = transformed[1]
    section = transformed[2]
    if 'DC' in abbrev:
        print(abbrev,x,y,section)
    continue
    com = CenterOfMass(
        prep_id=prep_id, structure=structure, x=x, y=y, section=section,
        created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned'
    )
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()



DC_L 581.2898481901691 466.8173868766281 49.457019038682986
DC_R 575.3076898031396 464.0790662020126 251.32001583909187


In [ ]:
# Print atlas COMs for comparison

from utilities.sqlcontroller import SqlController

def get_atlas_centers(
        atlas_box_size=(1000, 1000, 300),
        atlas_box_scales=(10, 10, 20),
        atlas_raw_scale=10
):
    atlas_box_scales = np.array(atlas_box_scales)
    atlas_box_size = np.array(atlas_box_size)
    atlas_box_center = atlas_box_size / 2
    sqlController = SqlController('Atlas')
    atlas_centers = sqlController.get_centers_dict('Atlas')

    for structure, center in atlas_centers.items():
        # transform into the atlas box coordinates that neuroglancer assumes
        center = atlas_box_center + np.array(center) * atlas_raw_scale / atlas_box_scales
        atlas_centers[structure] = center

    return atlas_centers

row_dict = get_atlas_centers()

In [ ]:
print(row_dict['3N_L'])
print(row_dict['3N_R'])

In [ ]:
11217/364.555

In [ ]:
R = np.array([[ 9.91558000e-01, -1.29661000e-01, -3.46808496e+03],
 [ 1.29661000e-01,  9.91558000e-01,  1.70894695e+03],
 [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])


In [ ]:
scale = 0.325
x = 350.0 
y = 494.0
T = np.array([[scale, 0, x],[0,scale, y],[0,0,1]])
tinv = np.linalg.inv(T)

In [ ]:
TRT = tinv @ R @ T
TRT[0][2] /= 32
TRT[1][2] /= 32
print(TRT)

In [ ]:
l = [1,2,3,'junk']
l[3]